## Item-based Collaborative Filtering

#### Rationale
1. Relatively large number of users compared to relevant news articles. Thus it is easier computationally to compare items than users.
2. Item stability > User stability. Once a news article is out, it's content is fixed, while a user might change taste often. This can make user-based collaborative filtering more inaccurate in relation to the user's present taste. Similarity between items is constanst, i.e. the need for recalculations will be less with item-based collaborative filtering.
3. Few news article interactions per user. This makes it harder to guess similar users as in user-based collaborative filtering.


#### Item-based collaborative filtering in a nutshell (MIND)
"Find articles that are likely to be of interest, based on shared user interest patterns. Return the top N articles that are most similar to any of the news articles the user has clicked on, based on the similarity calculations between items."
1. For each news article a user has clicked, get an overview of articles other users have also clicked
2. Matrix factorization for efficiency
3. Calculate the cosine similarity of each article - i.e. the article most often interacted with together with the initial one
4. Repeat steps for each news articles, and sort the recommendations list according to articles with the highest cosine similarity


#### Preperation of data

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import explode, split, col

spark.stop()

spark = SparkSession.builder \
    .appName("MINDItemBasedFiltering") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Define the schema of the dataset
schema = StructType([
    StructField("ImpressionID", IntegerType(), True),
    StructField("UserID", StringType(), True),
    StructField("Time", StringType(), True),
    StructField("History", StringType(), True),
    StructField("Impressions", StringType(), True)
])

# Load the dataset with the defined schema
data = spark.read.csv("data/MINDsmall_dev/behaviors.tsv", sep="\t", schema=schema)

data.show(5, truncate=False)

# Explode the history column into separate rows for each article per user 
# I.e. UserID | NewsArticle (that that user has stored in their history)
data = data.withColumn("NewsArticle", explode(split(col("History"), " "))) \
    .select(col("UserID").alias("user_id"), col("NewsArticle").alias("news_article"))

data.show(5, truncate=False)

+------------+------+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ImpressionID|UserID|Time                  |History                                                                                                                                                                                                                                                                                                  |Impressions                                                                            

#### Alternating Least Squares (ALS)

In [22]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import lit

# Prepare data to work with ALS
# Add a dummy 'rating' column to indicate interaction
data = data.withColumn("rating", lit(1))

# Index the user_id and news_article columns
user_indexer = StringIndexer(inputCol="user_id", outputCol="user_id_index").fit(data)
item_indexer = StringIndexer(inputCol="news_article", outputCol="news_article_id_index").fit(data)

data = user_indexer.transform(data)
data = item_indexer.transform(data)

# Select the final columns for ALS
data = data.select("user_id_index", "news_article_id_index", "rating")

data.show(5, truncate=False)

# Train the ALS model
# Note: We use implicitPrefs=True to indicate that we are working with implicit feedback (clicks)
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_index", itemCol="news_article_id_index", ratingCol="rating", coldStartStrategy="drop", implicitPrefs=True)
model = als.fit(data)

# Extract the item factors from the ALS model
#item_factors = model.itemFactors
item_factors = model.itemFactors.limit(100) #Subset for testing

item_factors.show(5)

+-------------+---------------------+------+
|user_id_index|news_article_id_index|rating|
+-------------+---------------------+------+
|10460.0      |6.0                  |1     |
|10460.0      |279.0                |1     |
|10460.0      |1243.0               |1     |
|10460.0      |201.0                |1     |
|10460.0      |1734.0               |1     |
+-------------+---------------------+------+
only showing top 5 rows

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.019209042, -0....|
| 10|[0.28451115, -0.0...|
| 20|[-0.014642524, 0....|
| 30|[0.16849972, -0.1...|
| 40|[0.10581791, -0.5...|
+---+--------------------+
only showing top 5 rows



#### Calculating Similarity - Locality-Sensitive Hasing (LSH)

In [23]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# In order to calculate the similarity between items by using Spark's LSH, we need to convert the item factors into a DenseVector
# Define a UDF that converts an array of floats into a DenseVector
to_vector = udf(lambda x: Vectors.dense(x), VectorUDT())

# Apply the UDF to the 'features' column
item_factors = item_factors.withColumn("features", to_vector("features"))

# Prepare for calculating similarity
# Initialize the LSH model
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0, numHashTables=3)

# Fit the LSH model on the item factors
model_lsh = brp.fit(item_factors)

# Transform item factors to hash table
item_factors_hashed = model_lsh.transform(item_factors)

# Calculate Similiary
# Calculate approx similarity join
similar_items = model_lsh.approxSimilarityJoin(item_factors_hashed, item_factors_hashed, threshold=1.5, distCol="EuclideanDistance")

# Show some results
similar_items.select(col("datasetA.id").alias("idA"), col("datasetB.id").alias("idB"), "EuclideanDistance").show()


Py4JJavaError: An error occurred while calling o1274.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 152.0 failed 1 times, most recent failure: Lost task 0.0 in stage 152.0 (TID 287) (10.24.29.162 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartiti